In [40]:
# Import the required libraries and dependencies
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import holoviews as hv
from holoviews import opts
from bokeh.layouts import column
from bokeh.plotting import show

In [2]:
# Read the CSV file into a Pandas DataFrame
cali_df = pd.read_csv('new_house_data.csv')

In [3]:
cali_df.head()

,Unnamed: 0,longitude,latitude,housing_median_age,population,households,median_income,median_house_value,ocean_proximity,avg_rooms_per_household,avg_bedrooms_per_household
0,0,-122.23,37.88,41.0,322.0,126.0,8.3252,452600.0,NEAR BAY,6.984127,1.023810
1,1,-122.22,37.86,21.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,6.238137,0.971880
2,2,-122.24,37.85,52.0,496.0,177.0,7.2574,352100.0,NEAR BAY,8.288136,1.073446
3,3,-122.25,37.85,52.0,558.0,219.0,5.6431,341300.0,NEAR BAY,5.817352,1.073059
4,4,-122.25,37.85,52.0,565.0,259.0,3.8462,342200.0,NEAR BAY,6.281853,1.081081


In [4]:
mode_value = cali_df['avg_bedrooms_per_household'].mode()[0]
 # Fill missing values with the mode 
cali_df['avg_bedrooms_per_household'].fillna(mode_value, inplace=True)

In [5]:
cali_df.count()

Unnamed: 0                    20640
longitude                     20640
latitude                      20640
housing_median_age            20640
population                    20640
households                    20640
median_income                 20640
median_house_value            20640
ocean_proximity               20640
avg_rooms_per_household       20640
avg_bedrooms_per_household    20640
dtype: int64

In [6]:
# scale house value wiht inflation 
cali_df['median_house_value']= cali_df['median_house_value'] * 1.69
cali_df = cali_df.drop(columns='Unnamed: 0')

In [7]:
# look at final df
cali_df.head()

,longitude,latitude,housing_median_age,population,households,median_income,median_house_value,ocean_proximity,avg_rooms_per_household,avg_bedrooms_per_household
0,-122.23,37.88,41.0,322.0,126.0,8.3252,764894.0,NEAR BAY,6.984127,1.023810
1,-122.22,37.86,21.0,2401.0,1138.0,8.3014,605865.0,NEAR BAY,6.238137,0.971880
2,-122.24,37.85,52.0,496.0,177.0,7.2574,595049.0,NEAR BAY,8.288136,1.073446
3,-122.25,37.85,52.0,558.0,219.0,5.6431,576797.0,NEAR BAY,5.817352,1.073059
4,-122.25,37.85,52.0,565.0,259.0,3.8462,578318.0,NEAR BAY,6.281853,1.081081


In [8]:
#columns to scale 
cali_df.columns


Index(['longitude', 'latitude', 'housing_median_age', 'population',
       'households', 'median_income', 'median_house_value', 'ocean_proximity',
       'avg_rooms_per_household', 'avg_bedrooms_per_household'],
      dtype='object')

In [9]:
#scale the data
columns_to_scale = ['housing_median_age', 'population',
       'households', 'median_income', 'median_house_value',
       'avg_rooms_per_household', 'avg_bedrooms_per_household']
scaler = StandardScaler()
cali_scaled = scaler.fit_transform(cali_df[columns_to_scale])

In [10]:
# creat scaled df
cali_scaled_df = pd.DataFrame(cali_scaled, columns=columns_to_scale)
cali_scaled_df.head()

,housing_median_age,population,households,median_income,median_house_value,avg_rooms_per_household,avg_bedrooms_per_household
0,0.982143,-0.974429,-0.977033,2.344766,2.129631,0.628559,-0.152553
1,-0.607019,0.861439,1.669961,2.332238,1.314156,0.327041,-0.262155
2,1.856182,-0.820777,-0.843637,1.782699,1.258693,1.155620,-0.047790
3,1.856182,-0.766028,-0.733781,0.932968,1.165100,0.156966,-0.048606
4,1.856182,-0.759847,-0.629157,-0.012881,1.172900,0.344711,-0.031676


In [11]:
# encode categorical variable proximity to ocean 
ocean_proximity_dummies = pd.get_dummies(cali_df['ocean_proximity'], dtype=int)
ocean_proximity_dummies.head()

,<1H OCEAN,INLAND,ISLAND,NEAR BAY,NEAR OCEAN
0,0,0,0,1,0
1,0,0,0,1,0
2,0,0,0,1,0
3,0,0,0,1,0
4,0,0,0,1,0


In [12]:
# concatenate dummies with scaled df
cali_scaled_df = pd.concat([cali_scaled_df, ocean_proximity_dummies], axis = 1)
cali_scaled_df.head()

,housing_median_age,population,households,median_income,median_house_value,avg_rooms_per_household,avg_bedrooms_per_household,<1H OCEAN,INLAND,ISLAND,NEAR BAY,NEAR OCEAN
0,0.982143,-0.974429,-0.977033,2.344766,2.129631,0.628559,-0.152553,0,0,0,1,0
1,-0.607019,0.861439,1.669961,2.332238,1.314156,0.327041,-0.262155,0,0,0,1,0
2,1.856182,-0.820777,-0.843637,1.782699,1.258693,1.155620,-0.047790,0,0,0,1,0
3,1.856182,-0.766028,-0.733781,0.932968,1.165100,0.156966,-0.048606,0,0,0,1,0
4,1.856182,-0.759847,-0.629157,-0.012881,1.172900,0.344711,-0.031676,0,0,0,1,0


## find best k value for model

In [13]:
#create list of k 
k = list(range(1,21))

In [14]:
inertia = [KMeans(n_clusters=i,random_state=7).fit(cali_scaled_df).inertia_ for i in k]
inertia

/Users/grantgorham/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/grantgorham/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/grantgorham/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/grantgorham/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: Fut

[158400.46579457365,
 133693.81572351672,
 112599.20714885852,
 93332.97430272878,
 82782.57247632979,
 74009.96020151577,
 67633.70960533014,
 62689.37228466217,
 58761.48568243565,
 55380.07966259047,
 52493.90165341168,
 49975.952599528966,
 48052.19228700086,
 46430.183286996995,
 44901.64284226256,
 43662.8373132304,
 42349.25050304145,
 41213.831540300045,
 40073.46602956686,
 38991.45641081751]

In [15]:
# Create a dictionary with the data to plot the Elbow curve
elbow_curve = {'k': k, 'inertia': inertia}

# Create a DataFrame with the data to plot the Elbow curve
elbow_plot_df = pd.DataFrame(elbow_curve)

elbow_plot_df.head()

,k,inertia
0,1,158400.465795
1,2,133693.815724
2,3,112599.207149
3,4,93332.974303
4,5,82782.572476


In [16]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_plot=elbow_plot_df.hvplot.line(
    x = 'k',
    y = 'inertia',
    xticks = k,
    label = 'Non-PCA elbow curve'
)
display(elbow_plot)

print('The optimal value for K is 4 based on the above figure using the elbow method')

:Curve   [k]   (inertia)

The optimal value for K is 4 based on the above figure using the elbow method


## Build model for clustering without using pca


In [17]:
# instantiate the model
model = KMeans(n_clusters = 4, random_state=42)

In [18]:
# Fit the K-Means model using the scaled data
model.fit(cali_scaled_df)

/Users/grantgorham/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=4, random_state=42)

In [19]:
# predict
cluster_labels = model.predict(cali_scaled_df)
print(cluster_labels)


[1 1 1 ... 0 0 0]


In [20]:
# create copy of df 
cali_scaled_df_copy = cali_scaled_df.copy()

In [21]:
# add labels to copy of df
cali_scaled_df_copy['cluster label'] = cluster_labels

In [22]:
cali_scaled_df_copy.head()

,housing_median_age,population,households,median_income,median_house_value,avg_rooms_per_household,avg_bedrooms_per_household,<1H OCEAN,INLAND,ISLAND,NEAR BAY,NEAR OCEAN,cluster label
0,0.982143,-0.974429,-0.977033,2.344766,2.129631,0.628559,-0.152553,0,0,0,1,0,1
1,-0.607019,0.861439,1.669961,2.332238,1.314156,0.327041,-0.262155,0,0,0,1,0,1
2,1.856182,-0.820777,-0.843637,1.782699,1.258693,1.155620,-0.047790,0,0,0,1,0,1
3,1.856182,-0.766028,-0.733781,0.932968,1.165100,0.156966,-0.048606,0,0,0,1,0,1
4,1.856182,-0.759847,-0.629157,-0.012881,1.172900,0.344711,-0.031676,0,0,0,1,0,1


In [23]:
cali_scaled_df_copy['cluster label'].value_counts()

cluster label
0    13054
1     5313
3     2197
2       76
Name: count, dtype: int64

In [44]:
def housing_scatter(dim1,dim2,cols):
    housing_scatter = cali_scaled_df_copy.hvplot.scatter(
        x = dim1,
        y=dim2,
        c = 'cluster label',
        cmap='Viridis',
        hover_cols = cols
    )
    return housing_scatter


In [37]:
housing_scatter('housing_median_age','median_income', ['median_house_value','avg_bedrooms_per_household'])

:Scatter   [housing_median_age]   (median_income,cluster label,median_house_value,avg_bedrooms_per_household)

In [41]:
def scatter3d(dim1, dim2, dim3):
    hv.extension('plotly')
    scatter = hv.Scatter3D(cali_scaled_df_copy,kdims=[dim1, dim2,dim3]).opts(color='cluster label', title='3d scatter')
    return scatter

In [45]:
scatter3d('median_house_value','median_income','avg_rooms_per_household')

<img src='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAEAAAABACAYAAACqaXHeAAAABHNCSVQICAgIfAhkiAAAAAlwSFlz
AAAB+wAAAfsBxc2miwAAABl0RVh0U29mdHdhcmUAd3d3Lmlua3NjYXBlLm9yZ5vuPBoAAA6zSURB
VHic7ZtpeFRVmsf/5966taWqUlUJ2UioBBJiIBAwCZtog9IOgjqACsogKtqirT2ttt069nQ/zDzt
tI4+CrJIREFaFgWhBXpUNhHZQoKBkIUASchWla1S+3ar7r1nPkDaCAnZKoQP/D7mnPOe9/xy76n3
nFSAW9ziFoPFNED2LLK5wcyBDObkb8ZkxuaoSYlI6ZcOKq1eWFdedqNzGHQBk9RMEwFAASkk0Xw3
ETacDNi2vtvc7L0ROdw0AjoSotQVkKSvHQz/wRO1lScGModBFbDMaNRN1A4tUBCS3lk7BWhQkgpD
lG4852/+7DWr1R3uHAZVQDsbh6ZPN7CyxUrCzJMRouusj0ipRwD2uKm0Zn5d2dFwzX1TCGhnmdGo
G62Nna+isiUqhkzuKrkQaJlPEv5mFl2fvGg2t/VnzkEV8F5ioioOEWkLG86fvbpthynjdhXYZziQ
x1hC9J2NFyi8vCTt91Fh04KGip0AaG9zuCk2wQCVyoNU3Hjezee9bq92duzzTmxsRJoy+jEZZZYo
GTKJ6SJngdJqAfRzpze0+jHreUtPc7gpBLQnIYK6BYp/uGhw9YK688eu7v95ysgshcg9qSLMo3JC
4jqLKQFBgdKDPoQ+Pltb8dUyQLpeDjeVgI6EgLIQFT5tEl3rn2losHVsexbZ3EyT9wE1uGdkIPcy
BGxn8QUq1QrA5nqW5i2tLqvrrM9NK6AdkVIvL9E9bZL/oyfMVd/jqvc8LylzRBKDJSzIExwhQzuL
QYGQj4rHfFTc8mUdu3E7yoLtbTe9gI4EqVgVkug2i5+uXGo919ixbRog+3fTbQ8qJe4ZOYNfMoTI
OoshUNosgO60AisX15aeI2PSIp5KiFLI9ubb1vV3Qb2ltwLakUCDAkWX7/nHKRmmGIl9VgYsUhJm
2NXjKYADtM1ygne9QQDIXlk49FBstMKx66D1v4+XuQr7vqTe0VcBHQlRWiOCbmmSYe2SqtL6q5rJ
zsTb7lKx3FKOYC4DoqyS/B5bvLPxvD9Qtf6saxYLQGJErmDOdOMr/zo96km1nElr8bmPOBwI9COv
HnFPRIwmkSOv9kcAS4heRsidOkpeWBgZM+UBrTFAXNYL5Vf2ii9c1trNzpYdaoVil3WIc+wdk+gQ
noie3ecCcxt9ITcLAPWt/laGEO/9U6PmzZkenTtsSMQ8uYywJVW+grCstAvCIaAdArAsIWkRDDs/
KzLm2YcjY1Lv0UdW73HabE9n6V66cxSzfEmuJssTpKGVp+0vHq73FwL46eOjpMpbRAnNmJFrGJNu
Ukf9Yrz+3rghiumCKNXXWPhLYcjxGsIpoCMsIRoFITkW8AuyM8jC1+/QLx4bozCEJIq38+1rtpR6
V/yzb8eBlRb3fo5l783N0CWolAzJHaVNzkrTzlEp2bQ2q3TC5gn6wpnoQAmwSiGh2GitnTmVMc5O
UyfKWUKCIsU7+fZDKwqdT6DDpvkzAX4/+AMFjk0tDp5GRXLpQ2MUmhgDp5gxQT8+Y7hyPsMi8uxF
71H0oebujHALECjFKaW9Lm68n18wXp2kVzIcABytD5iXFzg+WVXkegpAsOOYziqo0OkK76GyquC3
ltZAzMhhqlSNmmWTE5T6e3IN05ITFLM4GdN0vtZ3ob8Jh1NAKXFbm5PtLU/eqTSlGjkNAJjdgn/N
aedXa0tdi7+t9G0FIF49rtMSEgAs1kDLkTPO7ebm4IUWeyh1bKomXqlgMG6kJmHcSM0clYLJ8XtR
1GTnbV3F6I5wCGikAb402npp1h1s7LQUZZSMIfALFOuL3UUrfnS8+rez7v9qcold5tilgHbO1fjK
9ubb17u9oshxzMiUBKXWqJNxd+fqb0tLVs4lILFnK71H0Ind7uiPgACVcFJlrb0tV6DzxqqTIhUM
CwDf1/rrVhTa33/3pGPxJYdQ2l2cbgVcQSosdx8uqnDtbGjh9SlDVSMNWhlnilfqZk42Th2ZpLpf
xrHec5e815zrr0dfBZSwzkZfqsv+1FS1KUknUwPARVvItfKUY+cn57yP7qv07UE3p8B2uhUwLk09
e0SCOrK+hbdYHYLjRIl71wWzv9jpEoeOHhGRrJAzyEyNiJuUqX0g2sBN5kGK6y2Blp5M3lsB9Qh4
y2Ja6x6+i0ucmKgwMATwhSjdUu49tKrQ/pvN5d53ml2CGwCmJipmKjgmyuaXzNeL2a0AkQ01Th5j
2DktO3Jyk8f9vcOBQHV94OK+fPumJmvQHxJoWkaKWq9Vs+yUsbq0zGT1I4RgeH2b5wef7+c7bl8F
eKgoHVVZa8ZPEORzR6sT1BzDUAD/d9F78e2Tzv99v8D+fLVTqAKAsbGamKey1Mt9Ann4eH3gTXTz
idWtAJ8PQWOk7NzSeQn/OTHDuEikVF1R4z8BQCy+6D1aWRfY0tTGG2OM8rRoPaeIj5ZHzJxszElN
VM8K8JS5WOfv8mzRnQAKoEhmt8gyPM4lU9SmBK1MCQBnW4KONT86v1hZ1PbwSXPw4JWussVjtH9Y
NCoiL9UoH/6PSu8jFrfY2t36erQHXLIEakMi1SydmzB31h3GGXFDFNPaK8Rme9B79Ixrd0WN+1ij
NRQ/doRmuFLBkHSTOm5GruG+pFjFdAmorG4IXH1Qua6ASniclfFtDYt+oUjKipPrCQB7QBQ2lrgP
fFzm+9XWUtcqJ3/5vDLDpJ79XHZk3u8nGZ42qlj1+ydtbxysCezrydp6ugmipNJ7WBPB5tydY0jP
HaVNzs3QzeE4ZpTbI+ZbnSFPbVOw9vsfnVvqWnirPyCNGD08IlqtYkh2hjZ5dErEQzoNm+6ykyOt
Lt5/PQEuSRRKo22VkydK+vvS1XEKlhCJAnsqvcVvH7f/ZU2R67eXbMEGAMiIV5oWZWiWvz5Fv2xG
sjqNJQRvn3Rs2lji/lNP19VjAQDgD7FHhujZB9OGqYxRkZxixgRDVlqS6uEOFaJUVu0rPFzctrnF
JqijImVp8dEKVWyUXDk92zAuMZ6bFwpBU1HrOw6AdhQgUooChb0+ItMbWJitSo5Ws3IAOGEOtL53
0vHZih9sC4vtofZ7Qu6523V/fmGcds1TY3V36pUsBwAbSlxnVh2xLfAD/IAIMDf7XYIkNmXfpp2l
18rkAJAy9HKFaIr/qULkeQQKy9zf1JgDB2uaeFNGijo5QsUyacNUUTOnGO42xSnv4oOwpDi1zYkc
efUc3I5Gk6PhyTuVKaOGyLUAYPGIoY9Pu/atL/L92+4q9wbflRJ2Trpm/jPjdBtfnqB/dIThcl8A
KG7hbRuKnb8qsQsVvVlTrwQAQMUlf3kwJI24Z4JhPMtcfng5GcH49GsrxJpGvvHIaeem2ma+KSjQ
lIwUdYyCY8j4dE1KzijNnIP2llF2wcXNnsoapw9XxsgYAl6k+KzUXbi2yP3KR2ecf6z3BFsBICdW
nvnIaG3eHybqX7vbpEqUMT+9OL4Qpe8VON7dXuFd39v19FoAABRVePbGGuXTszO0P7tu6lghUonE
llRdrhArLvmKdh9u29jcFiRRkfLUxBiFNiqSU9icoZQHo5mYBI1MBgBH6wMNb+U7Pnw337H4gi1Y
ciWs+uks3Z9fztUvfzxTm9Ne8XXkvQLHNytOOZeiD4e0PgkAIAYCYknKUNUDSXEKzdWNpnil7r4p
xqkjTarZMtk/K8TQ6Qve78qqvXurGwIJqcOUKfUWHsm8KGvxSP68YudXq4pcj39X49uOK2X142O0
Tz5/u/7TVybqH0rSya6ZBwD21/gubbrgWdDgEOx9W

:Scatter3D   [median_house_value,median_income,avg_rooms_per_household]   (housing_median_age,population,households,avg_bedrooms_per_household,<1H OCEAN,INLAND,ISLAND,NEAR BAY,NEAR OCEAN,cluster label)